In [6]:
import spacy
import numpy as np
import pandas as pd
import nltk
import random
from fastapi import FastAPI
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from pydantic import BaseModel

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Sample training data (expandable)
training_data = {
    "hi|hello|hey": "Hello! How can I assist you today?",
    "track my order|where is my order": "Sure! Can you provide your order ID?",
    "refund status|money back|return money": "Refunds usually take 5-7 business days. Can you share your order ID?",
    "how to return an item|return policy": "You can return items within 30 days. Would you like a return label?",
    "talk to a human|customer service": "Connecting you to a support agent now...",
    "bye|goodbye|exit": "Goodbye! Have a great day!",
}

# Prepare training dataset
X_train = list(training_data.keys())
y_train = list(training_data.values())

# Convert text data into numerical vectors using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)

# Train Logistic Regression model
model = LogisticRegression()
model.fit(X_train_vectors, y_train)

# FastAPI application
app = FastAPI()

# Define request model
class UserQuery(BaseModel):
    query: str

@app.get("/")
def home():
    return {"message": "NLP Chatbot with Machine Learning is Running!"}

@app.post("/chat/")
def chat(user_query: UserQuery):
    query_vector = vectorizer.transform([user_query.query])
    response = model.predict(query_vector)[0]
    return {"user": user_query.query, "bot": response}

# Run the chatbot API using: uvicorn chatbot:app --reload
